In [3]:
import sys
sys.path.append('..')

In [4]:
from copy import deepcopy
from random import shuffle, choice
from library.solution import Solution
from library.algorithms.hill_climbing import hill_climbing

In previous notebooks, we defined the `Solution` class as an **abstract class** with methods that must be implemented by subclasses, depending on the specific optimization problem. While the implementation of solutions depend on the problem, all solutions share common principles: they require a **representation**, a **fitness function**, and a method for **random initialization**.

By extending this class, we can define solution classes specific to different optimization problems. For example, we created the `IntBinSolution` class to represent solutions for the IntBin optimization problem.

We then applied the Hill Climbing algorithm to the IntBin problem by further extending `IntBinSolution` to implement the `get_neighbors()` method, which is essential for Hill Climbing algorithm. To do this, we created a new class, `IntBinHillClimbingSolution`.

Today, we'll use Hill Climbing to solve two new problems: the Traveling Salesperson Problem (TSP) and the Knapsack Problem (KS).

## Traveling Salesperson Problem

**Description:** The Traveling Salesperson Problem (TSP) is the challenge of finding the shortest possible route that starts in a given city, visits each of the remaining N-1 cities exactly once, and returns to the starting city.

**Search space:** All possible permutations of city visit orders, forming valid round-trip routes.

**Representation:** List of city indexes that compose the route

**Fitness function:** f(x) = Total distance traveled, computed as the sum of distances between consecutive cities in the route.

**Goal:** Minimize f(x).

Let's begin by implementing the `TSPSolution` class, which inherits from `Solution`. As a result, it must implement the `fitness()` and `random_initial_representation()` methods.

This class represents a solution to the Traveling Salesperson Problem (TSP) and does not include any implementation related to the optimization algorithm that will be used to solve it.

![TSP Solution](images/tsp-solution.png)

In [7]:
from library.problems.data.tsp_data import distance_matrix

class TSPSolution(Solution):
    def __init__(self, repr=None, distance_matrix=distance_matrix, starting_idx=0):
        self.starting_idx = starting_idx
        self.distance_matrix = distance_matrix
        super().__init__(repr=repr)

    def random_initial_representation(self):
        # Route starts in starting idx
        route = [self.starting_idx]
        # Get city idx to visit and shuffle them
        idx_to_visit = [idx for idx in range(len(self.distance_matrix)) if idx != self.starting_idx]
        shuffle(idx_to_visit)
        # Add idx to visit to route
        route = route + idx_to_visit
        # Route ends in starting idx
        route = route + [self.starting_idx]
        return route

    def fitness(self):
        total_distance = 0
        for i in range(len(self.repr)-1):
            total_distance += self.distance_matrix[self.repr[i]][self.repr[i+1]]
        return total_distance


Let's test it

In [8]:
solution = TSPSolution()

print('Random solution:', solution)
print('Fitness:', solution.fitness())

Random solution: [0, 6, 7, 12, 11, 5, 8, 4, 1, 10, 9, 2, 3, 0]
Fitness: 15624


### Solving TSP with Hill Climbing

To use Hill Climbing to solve TSP we need to define a `TSPHillClimbingSolution` class that implements the `get_neighbors()` method. We also need to ensure that this function returns a list of solutions that also implement the `get_neighbors()` method, therefore, return a list of solutions of type `TSPHillClimbingSolution` too.

A TSP neighbor solution is obtained by swapping the positions of two consecutive cities in the route.

![TSP Hill Climbing Solution](images/tsp-hillclimbing-solution.png)

In [ ]:
class TSPHillClimbingSolution(TSPSolution):
    def get_neighbors(self):
        neighbors = []
        for i in range(1, len(self.repr)-2):
            new_route = deepcopy(self.repr)
            new_route[i], new_route[i+1] = new_route[i+1], new_route[i]
            neighbor = TSPHillClimbingSolution(
                repr=new_route,
                distance_matrix=self.distance_matrix,
                starting_idx=self.starting_idx
            )
            neighbors.append(neighbor)

        return neighbors

Let's test it

In [10]:
solution = TSPHillClimbingSolution()
print('Solution:', solution)

neighbors = solution.get_neighbors()
print('Neihghbors:')
for neighbor in neighbors:
    print(neighbor)

Solution: [0, 1, 12, 3, 5, 10, 11, 6, 7, 9, 4, 2, 8, 0]
Neihghbors:
[0, 12, 1, 3, 5, 10, 11, 6, 7, 9, 4, 2, 8, 0]
[0, 1, 3, 12, 5, 10, 11, 6, 7, 9, 4, 2, 8, 0]
[0, 1, 12, 5, 3, 10, 11, 6, 7, 9, 4, 2, 8, 0]
[0, 1, 12, 3, 10, 5, 11, 6, 7, 9, 4, 2, 8, 0]
[0, 1, 12, 3, 5, 11, 10, 6, 7, 9, 4, 2, 8, 0]
[0, 1, 12, 3, 5, 10, 6, 11, 7, 9, 4, 2, 8, 0]
[0, 1, 12, 3, 5, 10, 11, 7, 6, 9, 4, 2, 8, 0]
[0, 1, 12, 3, 5, 10, 11, 6, 9, 7, 4, 2, 8, 0]
[0, 1, 12, 3, 5, 10, 11, 6, 7, 4, 9, 2, 8, 0]
[0, 1, 12, 3, 5, 10, 11, 6, 7, 9, 2, 4, 8, 0]
[0, 1, 12, 3, 5, 10, 11, 6, 7, 9, 4, 8, 2, 0]


And now we can apply the hill climbing algorithm by passing it a random initial solution.

In [32]:
initial_solution = TSPHillClimbingSolution()
best_solution = hill_climbing(initial_solution, maximization=False, verbose=True)

print('Best solution:', best_solution)

Current solution: [0, 6, 5, 9, 11, 4, 3, 1, 12, 2, 7, 8, 10, 0] with fitness 17172
Neighbor: [0, 5, 6, 9, 11, 4, 3, 1, 12, 2, 7, 8, 10, 0] with fitness 17315
Neighbor: [0, 6, 9, 5, 11, 4, 3, 1, 12, 2, 7, 8, 10, 0] with fitness 16830
Neighbor: [0, 6, 5, 11, 9, 4, 3, 1, 12, 2, 7, 8, 10, 0] with fitness 17722
Neighbor: [0, 6, 5, 9, 4, 11, 3, 1, 12, 2, 7, 8, 10, 0] with fitness 17276
Neighbor: [0, 6, 5, 9, 11, 3, 4, 1, 12, 2, 7, 8, 10, 0] with fitness 17173
Neighbor: [0, 6, 5, 9, 11, 4, 1, 3, 12, 2, 7, 8, 10, 0] with fitness 17711
Neighbor: [0, 6, 5, 9, 11, 4, 3, 12, 1, 2, 7, 8, 10, 0] with fitness 17120
Neighbor: [0, 6, 5, 9, 11, 4, 3, 1, 2, 12, 7, 8, 10, 0] with fitness 19586
Neighbor: [0, 6, 5, 9, 11, 4, 3, 1, 12, 7, 2, 8, 10, 0] with fitness 17170
Neighbor: [0, 6, 5, 9, 11, 4, 3, 1, 12, 2, 8, 7, 10, 0] with fitness 18139
Neighbor: [0, 6, 5, 9, 11, 4, 3, 1, 12, 2, 7, 10, 8, 0] with fitness 17229
Current solution: [0, 6, 9, 5, 11, 4, 3, 1, 12, 2, 7, 8, 10, 0] with fitness 16830
Neighbor:

The implementations of `TSPSolution` and `TSPHillClimbingSolution` can be found in `library/problems/tsp.py`

## Knapsack Problem

**Description:** The Knapsack Problem involves selecting a subset of N items, each with a given value and weight, to pack into a container with a fixed capacity. If the total weight of selected items exceeds the capacity, the solution is invalid. The goal is to maximize the total value of items while ensuring they fit within the container's constraints.

**Search space:** All possible subsets of items that can be placed in the knapsack.

**Representation:** Binary string of length N (number of items), where 1 indicates the item is included in the knapsack and 0 indicates the item is excluded.

**Fitness function:** f(x)= Total value inside the knapsack. If the total size exceeds the knapsack's capacity, the solution is invalid and assigned a fitness of -inf.

**Goal:** Maximize f(x).

Similarly to what we've done for TSP, let's begin by implementing the `KSSolution` class, which inherits from `Solution` and implementes the `fitness()` and `random_initial_representation()` methods.

This class represents a solution to the Knapsack problem (KS) and does not include any implementation related to the optimization algorithm that will be used to solve it.

In [13]:
from library.problems.data.ks_data import values, weights, capacity

class KSSolution(Solution):
    def __init__(
        self,
        values: list[float] = values,
        weights: list[float] = weights,
        capacity: float = capacity,
        repr = None,
    ):
        self.values = values
        self.weights = weights
        self.capacity = capacity

        super().__init__(repr=repr)

    def random_initial_representation(self):
        repr = []
        for _ in range(len(self.values)):
            repr.append(choice([0, 1]))
        return repr
    
    def total_weight(self):
        total = 0
        for idx, bin_value in enumerate(self.repr):
            if bin_value == 1:
                total += self.weights[idx]
        return total

    def total_value(self):
        total = 0
        for idx, bin_value in enumerate(self.repr):
            if bin_value == 1:
                total += self.values[idx]
        return total
    
    def fitness(self):
        total_weight = self.total_weight()
        
        if total_weight > self.capacity:
            return -9999999999999999
        
        return self.total_value()

Let's test it

In [24]:
solution = KSSolution()

print(solution)
print(solution.fitness())

[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1]
-9999999999999999


### Solving KS with Hill Climbing

A neighbor solution is obtained by flipping a single bit, meaning adding one item to the knapsack, or removing one item from the knapsack.

Let's create the `KSHillClimbingSolution` that inherits from `KSSolution` and implements the `get_neighbors` method.


In [25]:
class KSHillClimbingSolution(KSSolution):
    def get_neighbors(self):
        neighbors = []

        for idx, bin_value in enumerate(self.repr):
            neighbor_repr = deepcopy(self.repr)
            if bin_value == 1:
                neighbor_repr[idx] = 0
            else:
                neighbor_repr[idx] = 1

            neighbor = KSHillClimbingSolution(
                repr=neighbor_repr,
                values=self.values,
                weights=self.weights,
                capacity=self.capacity,
            )
            neighbors.append(neighbor)

        return neighbors

Let's test it

In [26]:
solution = KSHillClimbingSolution()
print('Solution:', solution)

neighbors = solution.get_neighbors()
print('Neihghbors:')
for neighbor in neighbors:
    print(neighbor)

Solution: [1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]
Neihghbors:
[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]
[1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]
[1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]
[1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]
[1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]
[1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,

And now we can apply the hill climbing algorithm by passing it a random initial solution.

In [33]:
initial_solution = KSHillClimbingSolution()
best_solution = hill_climbing(initial_solution, maximization=True, max_iter=10, verbose=True)

print('Best solution:', best_solution)

Current solution: [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0] with fitness 3183
Neighbor: [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0] with fitness 2823
Neighbor: [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0] with fitness 3100
Neighbor: [1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0] with fitness 3124
Neighbor: [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0] with fitness 3053
Neighbor: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,

The implementations of `KSSolution` and `KSHillClimbingSolution` can be found in `library/problems/knapsack.py`